Εκφώνηση Ερωτήματος 1
*******
Το εργοστάσιο ῾῾Ουράνιο Τόξο᾿᾿, μεταξύ των διαφόρων χρωμάτων και σχετικών προϊόντων που παράγει,
παράγει και πέντε διαφορετικές ειδικές αποχρώσεις χρωμάτων (με ονομασίες: 1, 2, 3, 4 και 5) για
μεγάλους πελάτες της, για τις οποίες η ζήτηση είναι σταθερή. Κάθε τέτοια ειδική απόχρωση προκύπτει
από ένα συγκεκριμένο μηχάνημα ανάμιξης το οποίο θα πρέπει να καθαρίζεται πριν την παραγωγή της
επόμενης (διαφορετικής) απόχρωσης. Η διαδικασία ανάμιξης για κάθε απόχρωση από το 1 έως το 5
διαρκεί 40, 35, 45, 32 και 50 λεπτά, αντίστοιχα. Οι χρόνοι καθαρισμού του παραπάνω μηχανήματος
εξαρτώνται τόσο από τον τύπο όσο και από την απόχρωση των χρωμάτων. Για παράδειγμα, ένα χρώμα
που είναι βασισμένο σε λάδι απαιτεί παραπάνω χρόνο καθαρισμού απο ένα χρώμα που έχει βάση του το
νερό. Κάτι αντίστοιχο συμβαίνει όταν μετά απο μια απόχρωση σκούρου χρώματος θα πρέπει να παραχθεί
μια απόχρωση του λευκού. Ο χρόνος καθαρισμού του μηχανήματος (σε λεπτά) που απαιτούνται ώστε το
μηχάνημα να παράξει την απόχρωση j δεδομένου ότι η αμέσως προηγούμενη απόχρωση ήταν η i δίνεται
από το στοιχείο (i, j) του παρακάτω πίνακα.

Επειδή το εργοστάσιο έχει και άλλες δραστηριότητες παραγωγής, η διοίκησή του επιθυμεί η εβδομαδιαία παραγωγή των πέντε ειδικών αποχρώσεων να γίνεται σε όσο το δυνατόν μικρότερο χρόνο (αναμίξεις
και καθαρισμοί). Ποια σειρά παραγωγής των αποχρώσεων εξυπηρετεί βέλτιστα την παραπάνω επιθυμία
της διοίκησης; Η σειρά θα επαναλαμβάνεται κάθε εβδομάδα, επομένως ο χρόνος καθαρισμού μεταξύ της
τελευταίας παραγωγής απόχρωσης μιας εβδομάδας και της πρώτης παραγωγής απόχρωσης της επόμενης
εβδομάδας θα πρέπει να προσμετρηθεί στον συνολικό χρόνο καθαρισμού του μηχανήματος.


In [24]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import numpy as np

import shutil
import sys
import os.path

import gurobipy

from pyomo.environ import *
from pyomo.gdp import *

In [25]:
Mix_times = [40, 35, 45, 32, 50]

Clean_times = [
    [0, 11, 7, 13, 11],
    [5, 0, 13, 15, 15],
    [13, 15, 0, 23, 11],
    [9, 13, 5, 0, 3],
    [3, 7, 7, 7, 0]
]

TASKS = {
    'batch1': {'cleaning_time': Clean_times[0], 'mixing_time': Mix_times[0]},
    'batch2': {'cleaning_time': Clean_times[1], 'mixing_time': Mix_times[1]},
    'batch3': {'cleaning_time': Clean_times[2], 'mixing_time': Mix_times[2]},
    'batch4': {'cleaning_time': Clean_times[3], 'mixing_time': Mix_times[3]},
    'batch5': {'cleaning_time': Clean_times[4], 'mixing_time': Mix_times[4]}
}


succ = np.zeros_like(Clean_times)
for i in range(len(Clean_times)):
    for j in range(len(Clean_times)):
        if Clean_times[i][j] == 0:
            succ[i][j] = 0
        else:
            succ[i][j] = 1

In [26]:
#Initializing the Graph Class
class Graph:
    def __init__(self, vertices):
        self.V = vertices
        self.graph = []
        self.nodes = []
        self.MST = []

    def addEdge(self, s, d, w):
        self.graph.append([s, d, w])
    
    def addNode(self, value):
        self.nodes.append(value)

    def printSolution(self,s,d,w):
        for s, d, w in self.MST:
            print("%s - %s: %s" % (s, d, w))


In [27]:
#Implementing Disjoint Set data structure and its functions
class DisjointSet:
    def __init__(self, vertices):
        self.vertices = vertices
        self.parent = {}
        for v in vertices:
            self.parent[v] = v
        self.rank = dict.fromkeys(vertices, 0)
    
    def find(self, item):
        if self.parent[item] == item:
            return item
        else:
            return self.find(self.parent[item])
    
    def union(self, x, y):
        xroot = self.find(x)
        yroot = self.find(y)
        if self.rank[xroot] < self.rank[yroot]:
            self.parent[xroot] = yroot
        elif self.rank[xroot] > self.rank[yroot]:
            self.parent[yroot] = xroot
        else:
            self.parent[yroot] = xroot
            self.rank[xroot] += 1

In [28]:
#Function to implement Kruskal's Algorithm
def kruskalAlgo(self):
    i, e = 0, 0
    ds = dst.DisjointSet(self.nodes)
    self.graph = sorted(self.graph, key=lambda item: item[2])
    while e < self.V - 1:
        s, d, w = self.graph[i]
        i += 1
        x = ds.find(s)
        y = ds.find(d)
        if x != y:
            e += 1
            self.MST.append([s,d,w])
            ds.union(x,y)
    self.printSolution(s,d,w)

g = Graph(5)
g.addNode("A")
g.addNode("B")
g.addNode("C")
g.addNode("D")
g.addNode("E")
g.addEdge("A", "B", 5)
g.addEdge("A", "C", 13)
g.addEdge("A", "E", 15)
g.addEdge("B", "A", 5)
g.addEdge("B", "C", 10)
g.addEdge("B", "D", 8)
g.addEdge("C", "A", 13)
g.addEdge("C", "B", 10)
g.addEdge("C", "E", 20)
g.addEdge("C", "D", 6)
g.addEdge("D", "B", 8)
g.addEdge("D", "C", 6)
g.addEdge("E", "A", 15)
g.addEdge("E", "C", 20)

g.kruskalAlgo()

AttributeError: 'Graph' object has no attribute 'kruskalAlgo'